# Deploy <font color='red'>Census Model</font> Model Package from AWS Marketplace 




<font color='red'>Overview</font>

This sample notebook shows you how to deploy <font color='red'>Census Model</font> using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess** and **AmazonS3FullAccess** 
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to <font color='red'>Census Model</font>. 

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'>Census Model</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [4]:
import json
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage

In [ ]:
role = get_execution_role()
sagemaker_session = sage.Session()
boto3 = sagemaker_session.boto_session
bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
s3 = boto3.client("s3")
runtime = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [ ]:
model_name = "census-endpoint"
content_type = "application/json"
real_time_inference_instance_type = "ml.t2.medium"
batch_transform_inference_instance_type = "ml.m5.large"

### A. Create an endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

Once endpoint has been created, you would be able to perform real-time inference.

### B. Prepare input file for performing real-time inference

In [ ]:
# Sample input data
input_data = {
    "input": "UEsDBBQAAAAIAI5Si1ekm1Ws2wAAADsCAAALABQAX3JlbHMvLnJlbHMBABAAAAAAAAAAAAAAAAAAAAAAAK2SwWrDMAyG730K43ujtIMxRpNexqC3MroH8GwlMYktI6tb9vYzg7EFShlsR0n///EdtNvPYVKvyNlTbPSmqrXCaMn52Df6+fS4vtP7drV7wslIieTBp6xKJ+ZGDyLpHiDbAYPJFSWM5dIRByNl5B6SsaPpEbZ1fQv8k6HbBVMdXKP54DZand4T/o0NAcU4IwYsMa4TlzaLx1zghnuURjuyx7LOn4mqkDVcFtr+Xoi6zlt8IHsOGOWSF86C0aG7rmRSumZ0859Gy8S3zDzBG/H4QjR+ucDiB9rVB1BLAwQUAAAACACOUotXBCHWFboAAAAbAQAAEQAUAGRvY1Byb3BzL2NvcmUueG1sAQAQAAAAAAAAAAAAAAAAAAAAAABtjk1rhEAQRO/+Cpm7tm4gBFn1llMWAklgr0Pb0WGdD6Y7GX9+JrKYS45FvXrUedzsWn5TZONdr9q6USU59JNxc68+3p+rJzUOxRlDhz7Sa/SBohjiMu8cdxh6tYiEDoBxIau5zoTL5aePVkuOcYag8aZnglPTPIIl0ZMWDb/CKhxGdVdOeCjDV1x3wYRAK1lywtDWLfyxQtHyv4O9OciNzUGllOr0sHP5UQvXy8vbfr4yjkU7JAVD8QNQSwMEFAAAAAgAjlKLV/eOlC+MAAAA1wAAABAAFABkb2NQcm9wcy9hcHAueG1sAQAQAAAAAAAAAAAAAAAAAAAAAACdzs0KwjAQBOB7nyLk3qZ6ECn9uRTPHqr3kmzagNkNyVrq2xsRfACPwzAf0w67f4gNYnKEnTxUtRSAmozDpZO36VKe5dAX7TVSgMgOksgDTJ1cmUOjVNIr+DlVucbcWIp+5hzjoshap2Ek/fSArI51fVKwM6ABU4YfKL9is/G/qCH9+Zfu0ytkT/XFG1BLAwQUAAAACACOUotX50rGEvcAAACLAQAADwAUAHhsL3dvcmtib29rLnhtbAEAEAAAAAAAAAAAAAAAAAAAAAAAjY/NboMwEITveQpr78SkSiOKgKhqVYVr/3KsXLxgK9hGXqekb1+HiKjHnDyzXn07U2xPpmc/6Ek7W8JqmQJD2zipbVfCx/tLksG2WhSj84dv5w4srlvKfQkqhCHnnBqFRtDSDWjjX+u8ESFa33HXtrrBZ9ccDdrA79J0wz32IsRTpPRAcKHdwqLBo5CkEIPpLygjtIWqOKf61DhSdQ15tuy011a6sYTY6HfW62jGSe+1DCoWvn/YXGc71J0KcZilWQq8Kvg/+HR7fpkVBkt4m/SKJaxBS0f6qp92r49JrIgUgE2rtYxAYD7XUfharifwTONz5mrxB1BLAwQUAAAACACOUotXAcxbHt8AAACpAgAAGgAUAHhsL19yZWxzL3dvcmtib29rLnhtbC5yZWxzAQAQAAAAAAAAAAAAAAAAAAAAAACtks1qwzAQhO95CrH3WnZaSimRcwmFXNv0AYS8tkxsSWi3P3n7blNIYgihB5/EjLQzn5BW6+9xUJ+YqY/BQFWUoDC42PShM/C+e7l7gnW9WL3iYFmOkO8TKZkJZMAzp2etyXkcLRUxYZCdNubRssjc6WTd3naol2X5qPNlBtSTTLVtDORtU4HaHRL+Jzu2be9wE93HiIGvVGjyNmPzxlkuQxJsc4dsYGIXkgr6OsxyVhg+DHhJcdS36u/nrGeZxXP7Uf6Z1S2GhzkZvmLek0fkM8fJ+n0tWU4wevLj6sUPUEsDBBQAAAAIAI5Si1dUbVjl5AUAAMxUAAATABQAeGwvdGhlbWUvdGhlbWUxLnhtbAEAEAAAAAAAAAAAAAAAAAAAAAAA7Vzbbts4EH3vVwh6XbSybr4EcYq0qdEF0q6RZLHPtETZqilKS9Jpkq9fkpIoyZJzaZO2i44DJLwccYbDw6F54uT47U1GrGvMeJrTue2+GdkWplEep3Q9t/++Wrye2m9PXh2jI7HBGbYkmvIjNLc3QhRHjsMj2Yz4m7zAVPYlOcuQkFW2dmKGvspRMuJ4o9HYyVBK7ep59pjn8yRJI3yWR7sMU1EOwjBBQnrKN2nBbYuiDM/tdwTRrX1S+/iBYPUAVw0RYZeRdnwfGm9d9YOz9eo9YdY1InN7pF+2c3LsGAARfdxCvypcBYi3Xg93OlFfZjyvHK+PC331ZcbTABRFchJDPp56xnYLVBb7Y7vjD5P3XXxrfL+HH7tnU3/cwfsNPhiIxdnM9zr4oMGHA/hxEHzo4MMGPx7AB96p28Fr0IakdDu4giY6BpLk5OMgfLFowRuU0yJO+TwVB2iUoS85W8h+vbaSm9QStwVOUCRhHzG5xiKNkPUZ77Ayg44wegAQ8XsBzp7NLKU/3oHGptOOjo5VdihUSUrIpbgl+JxrZ3lO0nghG3VFP2MWptjIYmWtg1szpMsWy8U/qdhcblAhrbjawppXQ6+5VeRc0sE+OLZOFykV1Q6sN75EI/Epj8tmv50QzDC6tuZtQ74a4LHG/Mn3GXNL4COtueGwtfBea04rmnJTWEgdB+7YK01bPEIExyru5QD1srzgErmj1hptUIwHmlvzc72ZfD17NMMnOfE8QR71guz0dxOh3Zr1dW7PQi+0rQgVczuRqUEWs0KOx+nathBZy/M+EuUEH96LezOeDbPKHQWHot4xUTAuzhDflE/prvr4o43/XhioODzPBJxv9cKfuj/RC2d/aXGS4EgcaGmqsq8cZLD3+cHOkGer9eIXTvrBN23axlDwlMQRhEOJYzb7Phcek7xa5rzhGXth+Ng0VSCxsdQ3SfqURQSbo/0qv5Crb5kcaYm5/XpaFplpXEmfp63JqaF+1AkyHb38udsKtn8g2KPRywQ7HIh1eH+onf4WdVpv4XStd6fKV1+k7TP5BnFHyhZeyFpZWLL+Lj94derADqT8h5JGdQV6QmJFRyzf0bid6NuJTc3Cr3rr+azy+HbJFB0V3SxeRItUDn6OuFgihhQ11R1a/CW/JSSX88irkm1tcnY31K7w8hose23rK1Pz5f/uEMO2Rf6kXC3dVC9su8LalVW7QnfZ+5xoR6R3ulidW0wQXZVFRCNprzyurF3B0vVGmP1TnO5Evkir3F7OWK8Lbw6AGCdLGYcMsXM9oCxc6EJKY7kKpRF9IhLbkuArtLq8k1nODYLKFQ1RN4ZTDUPSppywurac03dsq7s38n1QStfLHY2Me/IMLKLSz2gZ9d4SOl3Eu5qO0VLw6mZYM6Tde5qIe3BV72onyXZ145TlyztTVDcgU/mcU+xUMbqQMVrpqawQx/KdHdYVTWoqcTJeNY/Kn4KlW6z6LnVJtsiouXr5dvUj212WZvmX8kmqdBKS3uGPTYDUd5or8repfXgrdtSODqy7HXaquZrn/gXzj4y+JmLgYqk6MBq4UKqOiFdp8TZ76KpZRrMuLitKXhO3eGEeusDDmofTX42HXV5UbKiI4QExgBiGGF5DDB+IAcQwxPAbYgRADCCGIUbQECMEYgAxDDHChhhjIAYQwxBj3BBjAsQAYhhiTBpiTIEYQAxDjGlDjBkQA4hhiDEr6nJLdOV1gdALnFhpfFNFtfxVwH5baayHlLE1bToAlZ/mt3dGxCd0T803QQCRfl+klzvRn9UifRBO3Fkt0lc9q3bPk0R60ZPoaa4k+uSnS/SQUv4vKQUkc+AFSOZADJDMgRggmQMxQDIHYoBkDsQAyRyIAZI5EAMkcyAGSObfLJkbpVzcPCSZu96kL5m3UFkqMLNImlV/VjKyf+PPpwv4dDp8Oh0+nQ5HFEjtQAyQ2oEYILUDMYAYILUDMUBqB2KA1A7EAKkdiAFSOxDjd5HaK4Xd6f/7mfpf1Jy8+g9QSwMEFAAAAAgAjlKLV1cPUbcEBQAATxcAABgAFAB4bC93b3Jrc2hlZXRzL3NoZWV0MS54bWwBABAAAAAAAAAAAAAAAAAAAAAAAJVYTXfaOBTd91f4eNHVDMZfspwaehpSQhKgOZ10Zu2AAJ8ai5EFtP9+ZNkx1rMO0bAAbN33ofeuL+Iln3/tc+tEWJnRYmS7g6FtkWJF11mxHdk/XqZ/Yvvz+ENypuxnuSOEWwJflDdsZO84P9w4TrnakX1aDuiBFGJtQ9k+5eKSbR262WQrckdXxz0puOMNh8hhJE+5iFXuskNp195MfJUHRtK1TGGf1672aVbY40Tee2bj5JBuyV+E/zg8M2uT8Rf6LG6IPdnOOHFa1DoTyVS7tRjZjOwv7s0SVwgJ+Dsj57Lz3ar2/Urpz+riYT2yRXnKHT3fs2w9zwpSyjtrskmPOa9uTmhOmRKz63IqNyTSaywE+p9szXcjGw/86tX6+k7PM5Jtd1z4igexba2OJaf79p5t0SPPRQZzciK5QMtEuveE7+qeyGNF81K+W/uskLb79Jf8PNfB3WAQin3x37ksVxOrycxtXNTGXmPsXYyHhqZ+Y+pfTD1D06AxDVpTzzRq2JiGl6j+wI3c2NAeNfaotQ8GfuiGyNA+auyjS3zXuNq4McaXfbsDZJ583NjHl+DY0LRqa02TYTdzHLqmO3dbol2YFlbZY+Mc3tjmep3q/y8Pb6RzO6wLjevvvhHPvTAPV+wxT+CNfy7ycdeJAYec+sGVynGX8nScMHq2pLpYlQh4w4EX9pVBRK9AXwRKXAuNLavHdZycxsPEOVVuG8RtH+GqiEkf4amIuz7CVxFf+4hARUz7iFBF3PcRSEXM+ohIRTz0EVhFPPYRsYp40lQMFHWugYCqLjQQUNalBnKpqyN40JLBMyKD1/Hn1/5AF267kKCGgDZMupCwhoA+3HlyEWmZUK9F8gGRjdf4A12710BA22YaCOjbQx/igb49aiCgb08aCOjbXNk+CLHoLsYh2Mayu4qCIED6fvttv4dX+u13UsV1qqAbt11IXEMAJSZdiPgd0Dbcr1ddncJ81TjwAKmmOgwIcq/DAKbMdBhQ4wcdBhJBh4FM0GEgFZTC+KC0C2W1TwZl+QobAiM2BH02gCfkNuixAdJ3EhiwIbjGBo0DH1R2qgsCNUHnB1R/psOAJ+BBhwF9etRhAIOfdBhQmLlSGMjvhbIaw+dwqRrHqMNZhQ2hERvCHht8UOHb8H02hJpNAxbfhdfYoHMAWDnVYALg516HAayahQbaoPMDWPUYGmiDzg9g3jy8qg3KaowA3ZbKMkZhEOjZgIzYgHpsCODJAPXYEMCTATLQBtTNG1Tta7PoXQ4HyEAJNJgAngx1fuD5ABn0HhkoATL4XVDKAPVvgdTfBUD2JTJUgsio91G/91AJovd7Hxn0PrqmBJHBKSEyOCVoMAE8LEYGShAZsCEyUILIgA3RVSWIrp8SIkMlwEZswH02wFMC7rEBMnSCDdiAr7EB95QAGygBNlACbKAE2OBMgA2UABv0XikDPMEs8HUlwO/13ukMDqrR6CJl26worZxs6gkia2eJnB7k5yvlghn1gIGkayLi1H8rN5Ty9sppR63Hg7XJWMmrQevyuH8l9XBCDl87o0p53U42rHKVypnHcGhbx5JMoYdqkMkyUnA5KB7ZB8o4SzNuW1XYb0zmtabn4mVHim8nwqqM6nynMs9xQtfr5uvHdH/4NJHvH/89Uv5pRvIT4dkqtZbkSP74TrbHPGX1moS5nvx4GsqX/P6cOBePiaPGctrZ+PjDf1BLAwQUAAAACACOUotXnIb+AtkBAABhBQAAFAAUAHhsL3NoYXJlZFN0cmluZ3MueG1sAQAQAAAAAAAAAAAAAAAAAAAAAAB9VMtu2zAQvOcrCN0binLd2IGsIPUDTVEFhu20aG60tLEI8KGSVBH9fckURQGunSN39jGzQ7K8e1WS/AbrhNGLjF3nGQHdmFbo0yJ7Omw+zLK76qp0zpNBi18DLM2g/SKbsoyEUu0WWed9f0upazpQ3F2bHnRAXoxV3IejPVHXW+Ct6wC8krTI809UcaGzqnSiKn21Vr00IwB55ApK6quSRuAv+EVYICvuEbCW0PhAmxzGHoGPgzqCJeaFrCAQakF7l+bU8JZzbugeXtNQpBD7fRbWdym4A8kjF9eJnnhD/ilK8962N8Y2O3AisGpQysEcedMY8uQwK39mDfcnFKqN9p0cydaCEoNK4WfRL02Lqr6aTpOVQXFWUJZH13JkDXCJd1GngfwjneSUzW+myMILa1q7hquj4Gn8JyATN9+wDiBLrrkV3JtLYgok5ozyDVLCYjWbz1D1vjcDNmw1cB1uNUhnNOo1v8QkGnGmIFKfhOFztMZlJ2SbBn8IKdGrQPt4QOvbW0G2XJ+QlvCEOOpYhylgUfQBqZ1RdhPvQIGfRGO00IjZ/XMa2YLn711QhnqHj6Yo4tRJinwXjTeW1Hw85/E0eozWvDPhu/Bkx9VFBuw/QsOfWV39AVBLAwQUAAAACACOUotXFP8DpOACAACzFgAADQAUAHhsL3N0eWxlcy54bWwBABAAAAAAAAAAAAAAAAAAAAAAAO1YW2/TMBR+36+w/M7s9DI2lGSCSgVeJqQNiVcncVJLjh057kj59fiSLGlp2co6tg4StbbP5Tvf8UU5cnjZlBzcUlUzKSIYnGIIqEhlxkQRwa838zfn8DI+CWu94vR6QakGxkHUEVxoXb1DqE4XtCT1qayoMJpcqpJoM1QFqitFSVZbp5KjEcZnqCRMwDgUy3Je6hqkcim0iXonAr75nEXQEPFgM5nRCH6kgirCIYpD1LrHYS5FjzKBXhCH9Q9wS7jBxdY8lVwqwERGG2pwz61MkJJ6m0+U31LNUgKu6JI6eA+yATU6HNT0fqgZ4SxR7BeMpIWwioNk6Ro7k4zzu5mcQi+Iw4poTZWYmwFo+zeryiyHkKKFcXb3WBeKrILR9OEOteQssyyK2XpeFxYjaaVkqWU7GWgA9eggfrGfPMp4nyiuMQuVSJWZ09ot1TnsRHHIaa6Nu2LFwrZaVhZfai1L08kYKaQg3AboPIaewB3xCOqFO6Ib+8lvMmQN2wgPsneWjsqDzI1dx/geez9R3vSv5xbsl1uwZ274kLkFT7tuz5ob/p/bn+5J/Ir3JH7Fe/LwubUd82lJKefXFu1bvlaaNflmWSa6LqkqvrpalglVc1es9dK59P7tyHy9et0HF7Ifv+esECUdOnxRUtNUu6oUGxKkM7HFqq1kDJyZBgi+K1Ld0EZ3H88mN3/DVHxiw5zGR5nUfoyDl8Z4ctFTHpuuYWK7Iwj8BrSDYO9Ugo1Ugsensg/78ZD96CjY4579ZMh+fNTsJy+WPf4nyeMjObPTo971Z0fN/u0xb/tXSP6Zzixqy7NBAbpWft5Jgb3PiuCVpcshaPI2xWTJuGbCj9Cw+jOYWdMXfk6rScLpepT+As04tLXzrB2qIvEXNKYTwTzH7rEOmxr/bNfs8sHY/rZrrG5XnF0MdvlY+S7NLm5JluJkqw+Z2nebZuzebZosse/v5w1tzD3q1gT118/xyU9QSwMEFAAAAAgAjlKLV9zcivOUAQAAuAYAABMAFABbQ29udGVudF9UeXBlc10ueG1sAQAQAAAAAAAAAAAAAAAAAAAAAACtlctuwjAQRff9iijbKjF0UVUVj0Vply1S6QeYeJIY4odsE8LfdxwKqpATQLBJlBmfe2fGTjKaNqKKajCWKzmOh+kgjkBminFZjOOfxUfyEk8nD6PFToONcK2047h0Tr8SYrMSBLWp0iAxkysjqMNHUxBNszUtgDwNBs8kU9KBdInzGvFkNIOcbioXvTcY3vsiHkdv+3XeahxTrSueUYdp4rMkyBmobA9YS3ZSXfJXWYpku8aWXNvHboeVhuLEgQvf2koXHYiWYcLHw8RS6CDh42Gi4HmQ8PEw4ToI10lolvfM1mfDnFB1D4dZDh1k3XsMArup8pxnwFS2EYikyM8M3fLOQTeVbW5ysNoAZbYEcKJK27u3+sI3yHAG0Zwa90kF6hJk5kZpi+ffQNpc29rhoHo60SgExnE4HtVeR5S+3vCkU/BTY8Au9G4qslVmvVRqfbN1YMipoFye8bclNcC+ncH9t3cv4p/2uTrcroK7F9CKnnF2+EGG/XV4s38rc8GWtxVa0t6Gd+76qH+og7Q/osnDL1BLAQI+ABQAAAAIAI5Si1ekm1Ws2wAAADsCAAALAAAAAAAAAAAAAAAAAAAAAABfcmVscy8ucmVsc1BLAQI+ABQAAAAIAI5Si1cEIdYVugAAABsBAAARAAAAAAAAAAAAAAAAABgBAABkb2NQcm9wcy9jb3JlLnhtbFBLAQI+ABQAAAAIAI5Si1f3jpQvjAAAANcAAAAQAAAAAAAAAAAAAAAAABUCAABkb2NQcm9wcy9hcHAueG1sUEsBAj4AFAAAAAgAjlKLV+dKxhL3AAAAiwEAAA8AAAAAAAAAAAAAAAAA4wIAAHhsL3dvcmtib29rLnhtbFBLAQI+ABQAAAAIAI5Si1cBzFse3wAAAKkCAAAaAAAAAAAAAAAAAAAAABsEAAB4bC9fcmVscy93b3JrYm9vay54bWwucmVsc1BLAQI+ABQAAAAIAI5Si1dUbVjl5AUAAMxUAAATAAAAAAAAAAAAAAAAAEYFAAB4bC90aGVtZS90aGVtZTEueG1sUEsBAj4AFAAAAAgAjlKLV1cPUbcEBQAATxcAABgAAAAAAAAAAAAAAAAAbwsAAHhsL3dvcmtzaGVldHMvc2hlZXQxLnhtbFBLAQI+ABQAAAAIAI5Si1echv4C2QEAAGEFAAAUAAAAAAAAAAAAAAAAAL0QAAB4bC9zaGFyZWRTdHJpbmdzLnhtbFBLAQI+ABQAAAAIAI5Si1cU/wOk4AIAALMWAAANAAAAAAAAAAAAAAAAANwSAAB4bC9zdHlsZXMueG1sUEsBAj4AFAAAAAgAjlKLV9zcivOUAQAAuAYAABMAAAAAAAAAAAAAAAAA+xUAAFtDb250ZW50X1R5cGVzXS54bWxQSwUGAAAAAAoACgCAAgAA1BcAAAAA"
}

<Add code snippet that shows the payload contents>

### C. Perform real-time inference

In [ ]:
# Invoke SageMaker endpoint
response = runtime.invoke_endpoint(
    EndpointName=model_name, ContentType=content_type, Body=json.dumps(input_data)
)

### D. Visualize output

In [ ]:
# Deserialize the response
response_body = response['Body'].read().decode('utf-8')
response_json = json.loads(response_body)

# Get predictions
print(response_json)

### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [ ]:
# upload the batch-transform job input files to S3
transform_input_folder = "census-model-transform-input"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name)
print("Transform input uploaded to " + transform_input)

In [ ]:
# Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

In [ ]:
# output is available on following path
transformer.output_path

## 4. Clean-up

### A. Delete the model

In [ ]:
model.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

